# Boolean Difference
## Initializations

In [11]:
import os
import re
import copy
from itertools import product
import subprocess

### Full adder circuit

In [12]:
stuck_node = 'G1'
SA_0_1 = 0
verilog_code = """
module circuit(
    A, B, Cin,sum, carry
);
    input A,B,Cin;
    output sum,carry;
    wire G1, G2, G3;
    xor gate1 (G1, A, B);
    and gate2 (G2, A, B);
    xor gate3 (sum, G1, Cin);

    and gate5 (G3, G1, Cin);
    or gate4 (carry, G2, G3);
endmodule
"""

### Pattern given in class

In [13]:
# stuck_node = 'w'
# SA_0_1 = 0
# verilog_code = """
# module circuit(x,y,z,f);
#     input x,y,z;
#     output f;
#     wire z,w,y1;
#     or call1 (f,u,w);
#     and call2 (u,x,y);
#     and call3 (w,y1,z);
#     not call4 (y1,y);
# endmodule
# """

### Non - Testable pattern

In [14]:
# stuck_node = 'z'
# SA_0_1 = 0
# verilog_code = """
# module circuit(x,y,z,f);
#     input x,y,z;
#     output f;
#     wire c,d;
#     and call1 (c,x,y);
#     and call2 (d,x,y,z);
#     or call3 (f,c,d);
# endmodule
# """

### Partial Credit Circuit

In [15]:
# stuck_node = 'z'
# SA_0_1 = 1
# verilog_code = """
# module circuit(a,b,c,d,e,f,z);
#     input a,b,c,d,e,f;
#     output z;
#     wire r,p,m,h,k,l,g,q,s,w,u;
#     xor call1 (z,r,u,w);
#     and call2 (r,a,p);
#     and call3 (u,a,m);
#     and call4 (w,q,s);
#     nor call5 (p,a,b,m);
#     or call6 (q,m,b);
#     or call7 (s,b,l);
#     or call8 (m,h,k);
#     and call9 (h,c,g);
#     and call10 (k,g,d);
#     xor call11 (l,e,f);
#     or call12 (g,c,d);

# endmodule
# """

### C432 circuit

In [16]:
# stuck_node = 'N1'
# SA_0_1 = 1
# output_node = "N388"
# verilog_code = """"
# module c432 (N1,N4,N8,N11,N14,N17,N21,N24,N27,N30,
#              N34,N37,N40,N43,N47,N50,N53,N56,N60,N63,
#              N66,N69,N73,N76,N79,N82,N86,N89,N92,N95,
#              N99,N102,N105,N108,N112,N115,N223,N329,N370,N421,
#              N430,N431,N432);

# input N1,N4,N8,N11,N14,N17,N21,N24,N27,N30,
#       N34,N37,N40,N43,N47,N50,N53,N56,N60,N63,
#       N66,N69,N73,N76,N79,N82,N86,N89,N92,N95,
#       N99,N102,N105,N108,N112,N115;

# output N223,N329,N370,N421,N430,N431,N432;

# wire N118,N119,N122,N123,N126,N127,N130,N131,N134,N135,
#      N138,N139,N142,N143,N146,N147,N150,N151,N154,N157,
#      N158,N159,N162,N165,N168,N171,N174,N177,N180,N183,
#      N184,N185,N186,N187,N188,N189,N190,N191,N192,N193,
#      N194,N195,N196,N197,N198,N199,N203,N213,N224,N227,
#      N230,N233,N236,N239,N242,N243,N246,N247,N250,N251,
#      N254,N255,N256,N257,N258,N259,N260,N263,N264,N267,
#      N270,N273,N276,N279,N282,N285,N288,N289,N290,N291,
#      N292,N293,N294,N295,N296,N300,N301,N302,N303,N304,
#      N305,N306,N307,N308,N309,N319,N330,N331,N332,N333,
#      N334,N335,N336,N337,N338,N339,N340,N341,N342,N343,
#      N344,N345,N346,N347,N348,N349,N350,N351,N352,N353,
#      N354,N355,N356,N357,N360,N371,N372,N373,N374,N375,
#      N376,N377,N378,N379,N380,N381,N386,N393,N399,N404,
#      N407,N411,N414,N415,N416,N417,N418,N419,N420,N422,
#      N425,N428,N429;

# not NOT1_1 (N118, N1);
# not NOT1_2 (N119, N4);
# not NOT1_3 (N122, N11);
# not NOT1_4 (N123, N17);
# not NOT1_5 (N126, N24);
# not NOT1_6 (N127, N30);
# not NOT1_7 (N130, N37);
# not NOT1_8 (N131, N43);
# not NOT1_9 (N134, N50);
# not NOT1_10 (N135, N56);
# not NOT1_11 (N138, N63);
# not NOT1_12 (N139, N69);
# not NOT1_13 (N142, N76);
# not NOT1_14 (N143, N82);
# not NOT1_15 (N146, N89);
# not NOT1_16 (N147, N95);
# not NOT1_17 (N150, N102);
# not NOT1_18 (N151, N108);
# nand NAND2_19 (N154, N118, N4);
# nor NOR2_20 (N157, N8, N119);
# nor NOR2_21 (N158, N14, N119);
# nand NAND2_22 (N159, N122, N17);
# nand NAND2_23 (N162, N126, N30);
# nand NAND2_24 (N165, N130, N43);
# nand NAND2_25 (N168, N134, N56);
# nand NAND2_26 (N171, N138, N69);
# nand NAND2_27 (N174, N142, N82);
# nand NAND2_28 (N177, N146, N95);
# nand NAND2_29 (N180, N150, N108);
# nor NOR2_30 (N183, N21, N123);
# nor NOR2_31 (N184, N27, N123);
# nor NOR2_32 (N185, N34, N127);
# nor NOR2_33 (N186, N40, N127);
# nor NOR2_34 (N187, N47, N131);
# nor NOR2_35 (N188, N53, N131);
# nor NOR2_36 (N189, N60, N135);
# nor NOR2_37 (N190, N66, N135);
# nor NOR2_38 (N191, N73, N139);
# nor NOR2_39 (N192, N79, N139);
# nor NOR2_40 (N193, N86, N143);
# nor NOR2_41 (N194, N92, N143);
# nor NOR2_42 (N195, N99, N147);
# nor NOR2_43 (N196, N105, N147);
# nor NOR2_44 (N197, N112, N151);
# nor NOR2_45 (N198, N115, N151);
# and AND9_46 (N199, N154, N159, N162, N165, N168, N171, N174, N177, N180);
# not NOT1_47 (N203, N199);
# not NOT1_48 (N213, N199);
# not NOT1_49 (N223, N199);
# xor XOR2_50 (N224, N203, N154);
# xor XOR2_51 (N227, N203, N159);
# xor XOR2_52 (N230, N203, N162);
# xor XOR2_53 (N233, N203, N165);
# xor XOR2_54 (N236, N203, N168);
# xor XOR2_55 (N239, N203, N171);
# nand NAND2_56 (N242, N1, N213);
# xor XOR2_57 (N243, N203, N174);
# nand NAND2_58 (N246, N213, N11);
# xor XOR2_59 (N247, N203, N177);
# nand NAND2_60 (N250, N213, N24);
# xor XOR2_61 (N251, N203, N180);
# nand NAND2_62 (N254, N213, N37);
# nand NAND2_63 (N255, N213, N50);
# nand NAND2_64 (N256, N213, N63);
# nand NAND2_65 (N257, N213, N76);
# nand NAND2_66 (N258, N213, N89);
# nand NAND2_67 (N259, N213, N102);
# nand NAND2_68 (N260, N224, N157);
# nand NAND2_69 (N263, N224, N158);
# nand NAND2_70 (N264, N227, N183);
# nand NAND2_71 (N267, N230, N185);
# nand NAND2_72 (N270, N233, N187);
# nand NAND2_73 (N273, N236, N189);
# nand NAND2_74 (N276, N239, N191);
# nand NAND2_75 (N279, N243, N193);
# nand NAND2_76 (N282, N247, N195);
# nand NAND2_77 (N285, N251, N197);
# nand NAND2_78 (N288, N227, N184);
# nand NAND2_79 (N289, N230, N186);
# nand NAND2_80 (N290, N233, N188);
# nand NAND2_81 (N291, N236, N190);
# nand NAND2_82 (N292, N239, N192);
# nand NAND2_83 (N293, N243, N194);
# nand NAND2_84 (N294, N247, N196);
# nand NAND2_85 (N295, N251, N198);
# and AND9_86 (N296, N260, N264, N267, N270, N273, N276, N279, N282, N285);
# not NOT1_87 (N300, N263);
# not NOT1_88 (N301, N288);
# not NOT1_89 (N302, N289);
# not NOT1_90 (N303, N290);
# not NOT1_91 (N304, N291);
# not NOT1_92 (N305, N292);
# not NOT1_93 (N306, N293);
# not NOT1_94 (N307, N294);
# not NOT1_95 (N308, N295);
# not NOT1_96 (N309, N296);
# not NOT1_97 (N319, N296);
# not NOT1_98 (N329, N296);
# xor XOR2_99 (N330, N309, N260);
# xor XOR2_100 (N331, N309, N264);
# xor XOR2_101 (N332, N309, N267);
# xor XOR2_102 (N333, N309, N270);
# nand NAND2_103 (N334, N8, N319);
# xor XOR2_104 (N335, N309, N273);
# nand NAND2_105 (N336, N319, N21);
# xor XOR2_106 (N337, N309, N276);
# nand NAND2_107 (N338, N319, N34);
# xor XOR2_108 (N339, N309, N279);
# nand NAND2_109 (N340, N319, N47);
# xor XOR2_110 (N341, N309, N282);
# nand NAND2_111 (N342, N319, N60);
# xor XOR2_112 (N343, N309, N285);
# nand NAND2_113 (N344, N319, N73);
# nand NAND2_114 (N345, N319, N86);
# nand NAND2_115 (N346, N319, N99);
# nand NAND2_116 (N347, N319, N112);
# nand NAND2_117 (N348, N330, N300);
# nand NAND2_118 (N349, N331, N301);
# nand NAND2_119 (N350, N332, N302);
# nand NAND2_120 (N351, N333, N303);
# nand NAND2_121 (N352, N335, N304);
# nand NAND2_122 (N353, N337, N305);
# nand NAND2_123 (N354, N339, N306);
# nand NAND2_124 (N355, N341, N307);
# nand NAND2_125 (N356, N343, N308);
# and AND9_126 (N357, N348, N349, N350, N351, N352, N353, N354, N355, N356);
# not NOT1_127 (N360, N357);
# not NOT1_128 (N370, N357);
# nand NAND2_129 (N371, N14, N360);
# nand NAND2_130 (N372, N360, N27);
# nand NAND2_131 (N373, N360, N40);
# nand NAND2_132 (N374, N360, N53);
# nand NAND2_133 (N375, N360, N66);
# nand NAND2_134 (N376, N360, N79);
# nand NAND2_135 (N377, N360, N92);
# nand NAND2_136 (N378, N360, N105);
# nand NAND2_137 (N379, N360, N115);
# nand NAND4_138 (N380, N4, N242, N334, N371);
# nand NAND4_139 (N381, N246, N336, N372, N17);
# nand NAND4_140 (N386, N250, N338, N373, N30);
# nand NAND4_141 (N393, N254, N340, N374, N43);
# nand NAND4_142 (N399, N255, N342, N375, N56);
# nand NAND4_143 (N404, N256, N344, N376, N69);
# nand NAND4_144 (N407, N257, N345, N377, N82);
# nand NAND4_145 (N411, N258, N346, N378, N95);
# nand NAND4_146 (N414, N259, N347, N379, N108);
# not NOT1_147 (N415, N380);
# and AND8_148 (N416, N381, N386, N393, N399, N404, N407, N411, N414);
# not NOT1_149 (N417, N393);
# not NOT1_150 (N418, N404);
# not NOT1_151 (N419, N407);
# not NOT1_152 (N420, N411);
# nor NOR2_153 (N421, N415, N416);
# nand NAND2_154 (N422, N386, N417);
# nand NAND4_155 (N425, N386, N393, N418, N399);
# nand NAND3_156 (N428, N399, N393, N419);
# nand NAND4_157 (N429, N386, N393, N407, N420);
# nand NAND4_158 (N430, N381, N386, N422, N399);
# nand NAND4_159 (N431, N381, N386, N425, N428);
# nand NAND4_160 (N432, N381, N422, N425, N429);

# endmodule
# """


### Parser of Verilog circuit:

In [17]:
def parse_verilog(verilog_code):
    module_info = {"name": "", "inputs": [], "outputs": [], "wires": [], "gates": []}

    # Patterns for parsing the Verilog code
    module_pattern = r"module\s+(\w+)\s*\("
    input_pattern = r"input(?:\s+wire)?\s+([\w\s,]+);"
    output_pattern = r"output(?:\s+wire)?\s+([\w\s,]+);"
    wire_pattern = r"wire\s+([\w\s,]+);"
    gate_pattern = r"(\w+)\s+\w+\s*\((\w+),\s*([\w,\s]+)\);"


    # Parse module name
    module_match = re.search(module_pattern, verilog_code)
    if module_match:
        module_info["name"] = module_match.group(1)

    # Parse inputs
    input_match = re.search(input_pattern, verilog_code, re.DOTALL)
    if input_match:
        module_info["inputs"] = [inp.strip() for inp in input_match.group(1).split(",") if inp.strip()]

    # Parse outputs
    output_match = re.search(output_pattern, verilog_code, re.DOTALL)
    if output_match:
        module_info["outputs"] = [out.strip() for out in output_match.group(1).split(",") if out.strip()]

    # Parse wires
    wire_match = re.search(wire_pattern, verilog_code, re.DOTALL)
    if wire_match:
        module_info["wires"] = [wire.strip() for wire in wire_match.group(1).split(",") if wire.strip()]

    # Parse gates
    for gate_match in re.finditer(gate_pattern, verilog_code):
        gate_type = gate_match.group(1)
        gate_output = gate_match.group(2)
        gate_inputs = [inp.strip() for inp in gate_match.group(3).split(",") if inp.strip()]
        module_info["gates"].append({
            "type": gate_type,
            "output": gate_output,
            "inputs": gate_inputs
        })

    return module_info

parsed_circuit = parse_verilog(verilog_code)
print(parsed_circuit)


{'name': 'circuit', 'inputs': ['A', 'B', 'Cin'], 'outputs': ['sum', 'carry'], 'wires': ['G1', 'G2', 'G3'], 'gates': [{'type': 'xor', 'output': 'G1', 'inputs': ['A', 'B']}, {'type': 'and', 'output': 'G2', 'inputs': ['A', 'B']}, {'type': 'xor', 'output': 'sum', 'inputs': ['G1', 'Cin']}, {'type': 'and', 'output': 'G3', 'inputs': ['G1', 'Cin']}, {'type': 'or', 'output': 'carry', 'inputs': ['G2', 'G3']}]}


### Output expression generator

In [18]:
def backtrack_for_stuck_node(parsed_circuit, stuck_node, select , output_node):
    boolean_expressions = {}
    

    for gate in parsed_circuit['gates']:
        gate_type = gate['type'].lower()
        gate_output = gate['output']
        gate_inputs = gate['inputs']

        if gate_type == "and":
            boolean_expressions[gate_output] = f"({' & '.join(gate_inputs)})"
        elif gate_type == "or":
            boolean_expressions[gate_output] = f"({' | '.join(gate_inputs)})"
        elif gate_type == "nor":
            boolean_expressions[gate_output] = f"~({' | '.join(gate_inputs)})"
        elif gate_type == "nand":
            boolean_expressions[gate_output] = f"~({' & '.join(gate_inputs)})"
        elif gate_type == "xor":
            boolean_expressions[gate_output] = f"({' ^ '.join(gate_inputs)})"
        elif gate_type == "xnor":
            boolean_expressions[gate_output] = f"~({' ^ '.join(gate_inputs)})"
        elif gate_type == "not" and len(gate_inputs) == 1:
            boolean_expressions[gate_output] = f"(~({gate_inputs[0]}))"
        elif gate_type == "buf" and len(gate_inputs) == 1:
            boolean_expressions[gate_output] = f"({gate_inputs[0]})"
        else:
            boolean_expressions[gate_output] = "Unsupported Gate Type"
    

    def resolve_expression(node):
        if node in parsed_circuit['inputs'] or node == stuck_node:
            return node
        
        if node in boolean_expressions:
            expr = boolean_expressions[node]
            for intermediate_node in boolean_expressions:
                if intermediate_node in expr:
                    expr = expr.replace(intermediate_node, resolve_expression(intermediate_node))
            return expr
        return node

    resolved_expressions = []
    
    if(select):
        for output in parsed_circuit['outputs']:
            resolved_output = resolve_expression(output)
            resolved_expressions.append(f"{resolved_output}")
    else:
        resolved_output = resolve_expression(output_node)
        resolved_expressions.append(f"{resolved_output}")
    
    return resolved_expressions
output_node = " "
select = 1
resolved_expressions = backtrack_for_stuck_node(parsed_circuit, stuck_node,select,output_node)
print("Expressions for outputs :")
i = 0
for out in parsed_circuit["outputs"]:
    print(out ,"Expression: ", resolved_expressions[i])
    i = i+1
#here the resolved expression only contains equations that have the stuck_node in the final expression. If there is no expression then the expression is removed from the list.
parsed_stuck_node = copy.deepcopy(parsed_circuit)
parsed_stuck_node['outputs'] = [stuck_node]
primary_input = parsed_stuck_node['inputs'][1]
stuck_node_expression = backtrack_for_stuck_node(parsed_stuck_node, primary_input,select,output_node)[0]
print(f"Stuck Node {stuck_node} expression:")
print(stuck_node_expression)


Expressions for outputs :
sum Expression:  (G1 ^ Cin)
carry Expression:  ((A & B) | (G1 & Cin))
Stuck Node G1 expression:
(A ^ B)


### Generate Stuck at fault expressions

In [19]:
def generate_stuck_at_fault_expressions(resolved_expressions, stuck_node, parsed_circuit, SA_0_1, stuck_node_expression):
    boolean_differences = []

    # Use word boundaries to ensure that 'stuck_node' is not part of another identifier
    stuck_node_regex = r'(?<!\w)' + re.escape(stuck_node) + r'(?!\w)'

    for output, expr in zip(parsed_circuit['outputs'], resolved_expressions):
        if stuck_node not in expr:
        #     print("output is not dependent on given input so no need to evaluate this expression")
            continue
        
        # Replace 'stuck_node' with '0' and '1', ensuring it matches as a whole word only
        exp_stuck_at_0 = re.sub(stuck_node_regex, '0', expr)
        exp_stuck_at_1 = re.sub(stuck_node_regex, '1', expr)
        
        if SA_0_1:
            boolean_difference = f"Boolean difference of {output} = ~({stuck_node_expression}) & (({exp_stuck_at_1}) ^ ({exp_stuck_at_0}))"
        else:
            boolean_difference = f"Boolean difference of {output} = {stuck_node_expression} & (({exp_stuck_at_1}) ^ ({exp_stuck_at_0}))"
        
        boolean_differences.append((output, boolean_difference))
    
    return boolean_differences

boolean_differences = generate_stuck_at_fault_expressions(resolved_expressions, stuck_node, parsed_circuit, SA_0_1, stuck_node_expression)
print(boolean_differences)

[('sum', 'Boolean difference of sum = (A ^ B) & (((1 ^ Cin)) ^ ((0 ^ Cin)))'), ('carry', 'Boolean difference of carry = (A ^ B) & ((((A & B) | (1 & Cin))) ^ (((A & B) | (0 & Cin))))')]


In [20]:
def generate_verilog(order, test_cases, true_expression, faulty_expression, stuck_node, sa_0_1):
    # Map variables from `order` list into module instantiations
    module_mapping = ", ".join([f".{var}({var})" for var in order])

    # Initialize test cases array
    test_cases_init = "\n".join(
        [f"        test_cases[{i}] = {len(order)}'b{tc};" for i, tc in enumerate(test_cases)]
    )

    # Assign inputs from test cases to individual signals
    input_assignment = "\n".join(
        [f"            {var} = inputs[{len(order) - idx - 1}];" for idx, var in enumerate(order)]
    )

    # Generate the Verilog code
    verilog_code = f"""
module true_circuit({', '.join(order)}, true_output);
    input {', '.join(order)};
    output true_output;
    assign true_output = {true_expression};
endmodule

module faulty_circuit({', '.join(order)}, fault_output);
    input {', '.join(order)};
    output fault_output;
    wire {stuck_node};
    assign {stuck_node} = {sa_0_1};
    assign fault_output = {faulty_expression};
endmodule

module test_bench();
    reg [{len(order)-1}:0] test_cases [0:{len(test_cases)-1}];
    reg [{len(order)-1}:0] inputs;
    reg {', '.join(order)};
    wire true_output, fault_output;

    true_circuit true_instance ({module_mapping}, .true_output(true_output));
    faulty_circuit faulty_instance ({module_mapping}, .fault_output(fault_output));

    integer i;
    initial begin


        // Initialize test cases
{test_cases_init}

        // Apply test cases
        for (i = 0; i <= {len(test_cases)-1}; i = i + 1) begin
            inputs = test_cases[i];
{input_assignment}
            #1; // Wait for outputs to stabilize
            $display("Test case = %b, True Output = %b, Faulty Output = %b", inputs, true_output, fault_output);
        end
        $finish;
    end
endmodule
"""

    # Write the Verilog code to a file
    file_name = "generated_verilog.v"
    with open(file_name, "w") as f:
        f.write(verilog_code)
    # print(f"Verilog code has been written to {file_name}")


def extract_nodes(expression):
    # Regular expression to match any alphanumeric sequence (potential node names)
    node_pattern = r'\b\w+\b'
    
    # Find all matches in the expression
    potential_nodes = re.findall(node_pattern, expression)
    
    # Filter out constants and operators (e.g., '0', '1', '&', '~', etc.)
    valid_nodes = [node for node in potential_nodes if node not in ['0', '1', '&', '|', '~']]
    
    # Return unique nodes
    return list(set(valid_nodes))

for output, boolean_difference in boolean_differences:
    print(f"{boolean_difference}")
    variables = extract_nodes(boolean_difference.split('= ')[1])

    if(len(variables)<= 20):
        module_code = (
            "module LogicalExpression(\n"
            + "    input " + ", ".join(variables) + ",\n"
            + "    output result\n);\n"
            + f"    assign result = {boolean_difference.split('= ')[1]};\n"
            + "endmodule\n"
        )

        # Generating Verilog testbench code
# Generating Verilog testbench code
        testbench_code = (
            "module Testbench();\n"
            + "    reg " + ", ".join(variables) + ";\n"
            + "    wire result;\n\n"
            + "    // Instantiate the module\n"
            + "    LogicalExpression dut (\n"
            + "        ." + ", .".join([f"{name}({name})" for name in variables]) + ",\n"
            + "        .result(result)\n"
            + "    );\n\n"
            + "    // Test all possible binary combinations\n"
            + "    integer i;\n"
            + "    reg found_pattern;\n\n"  # Flag to track if a pattern is found
            + "    initial begin\n"
            + "        found_pattern = 0;\n"  # Initialize the flag to 0
            # + '        $display("Checking for test cases where output is 1:");\n'
            + f"        for (i = 0; i < {2**len(variables)}; i = i + 1) begin\n"
            + "            { " + ", ".join(variables) + " } = i;\n"
            + "            #1; // Small delay for each evaluation\n"
            + "            if (result) begin\n"
            + "                found_pattern = 1;\n"  # Set the flag if a pattern is found
            + '                $display("%b,", { ' + ", ".join(variables) + " });\n"
            + "            end\n"
            + "        end\n"
            + "        if (!found_pattern) begin\n"
            + "        end\n"
            + "    end\n"
            + "endmodule\n"
        )

        # Writing the Verilog code to files
        with open("LogicalExpression.v", "w") as module_file:
            module_file.write(module_code)

        with open("LogicalExpression.v", "a") as testbench_file:
            testbench_file.write(testbench_code)

        # os.system("iverilog -o run LogicalExpression.v")
        # os.system("vvp run")
        compile_cmd = f"iverilog -o run LogicalExpression.v"
        subprocess.run(compile_cmd, shell=True, check=True, text=True)

        # Run the compiled Verilog file
        run_cmd = "vvp run"
        result = subprocess.run(run_cmd, shell=True, check=True, text=True, stdout=subprocess.PIPE)

        # Capture and process the output
        output_lines = result.stdout.strip().splitlines()  # Split into lines
        processed_output = [line.rstrip(',') for line in output_lines]  # Remove trailing commas
        # print(processed_output)
        # print()


        ##checker
        select = 0
        output_node = output
        parsed_stuck_node = copy.deepcopy(parsed_circuit)
        parsed_stuck_node['outputs'] = [stuck_node]
        primary_input = parsed_stuck_node['inputs'][0]
        base_expressions = backtrack_for_stuck_node(parsed_circuit, primary_input,select,output_node)
        # true_expression = base_expressions[0]
        # print()
        # print()
        # print(resolved_expressions)
        # print()
        # print()
        faulty_base_expression= backtrack_for_stuck_node(parsed_circuit,stuck_node,select,output_node)
        true_expression = base_expressions[0]
        print("output expressions:")
        print(true_expression)
        print("patterns that satisfy the given condition are:")
        print((variables),"rest all are x(don't cares)")
        if stuck_node in variables:
                # Replace occurrences of stuck_node with stuck_node_new
                stuck_node_regex = r'(?<!\w)' + re.escape(stuck_node) + r'(?!\w)'
                faulty_expression = re.sub(stuck_node_regex, f'{stuck_node}_new', faulty_base_expression[0])
                stuck_node = f"{stuck_node}_new"
        else:
            faulty_expression = faulty_base_expression[0]

        # print(faulty_expression)
        # print(len(processed_output))
        # Generate Verilog file
        if(len(processed_output)):
            generate_verilog(variables, processed_output, true_expression, faulty_expression, stuck_node, SA_0_1)
            compile_cmd = f"iverilog -o run generated_verilog.v"
            subprocess.run(compile_cmd, shell=True, check=True, text=True)

            # Run the compiled Verilog file
            run_cmd = "vvp run"
            result = subprocess.run(run_cmd, shell=True, check=True, text=True, stdout=subprocess.PIPE)
            raw_output = result.stdout.strip()  # Remove leading/trailing whitespace
            output_lines = raw_output.split("\n")  # Split into lines
            # print(output_lines)
            for i in range(0,len(output_lines)):
                print(output_lines[i])
        else:
            print("no pattern detected")
    else:
        # print(f"running this code is computationally expensive since the output is dependent on {len(variables)} input parameters\n")
        module_code = (
            "module LogicalExpression(\n"
            + "    input " + ", ".join(variables) + ",\n"
            + "    output result\n);\n"
            + f"    assign result = {boolean_difference.split('= ')[1]};\n"
            + "endmodule\n"
        )

        # Generating Verilog testbench code
# Generating Verilog testbench code
        testbench_code = (
            "module Testbench();\n"
            + "    reg " + ", ".join(variables) + ";\n"
            + "    wire result;\n\n"
            + "    // Instantiate the module\n"
            + "    LogicalExpression dut (\n"
            + "        ." + ", .".join([f"{name}({name})" for name in variables]) + ",\n"
            + "        .result(result)\n"
            + "    );\n\n"
            + "    // Test all possible binary combinations\n"
            + "    integer i;\n"
            + "    reg found_pattern;\n\n"  # Flag to track if a pattern is found
            + "    initial begin\n"
            + "        found_pattern = 0;\n"  # Initialize the flag to 0
            # + '        $display("Checking for test cases where output is 1:");\n'
            + f"        for (i = 0; i < 1000; i = i + 1) begin\n"
            + "            { " + ", ".join(variables) + " } = i;\n"
            + "            #1; // Small delay for each evaluation\n"
            + "            if (result) begin\n"
            + "                found_pattern = 1;\n"  # Set the flag if a pattern is found
            + '                $display("%b,", { ' + ", ".join(variables) + " });\n"
            + "            end\n"
            + "        end\n"
            + "        if (!found_pattern) begin\n"
            + "        end\n"
            + "    end\n"
            + "endmodule\n"
        )

        # Writing the Verilog code to files
        with open("LogicalExpression.v", "w") as module_file:
            module_file.write(module_code)

        with open("LogicalExpression.v", "a") as testbench_file:
            testbench_file.write(testbench_code)

        # os.system("iverilog -o run LogicalExpression.v")
        # os.system("vvp run")
        compile_cmd = f"iverilog -o run LogicalExpression.v"
        subprocess.run(compile_cmd, shell=True, check=True, text=True)

        # Run the compiled Verilog file
        run_cmd = "vvp run"
        result = subprocess.run(run_cmd, shell=True, check=True, text=True, stdout=subprocess.PIPE)

        # Capture and process the output
        output_lines = result.stdout.strip().splitlines()  # Split into lines
        processed_output = [line.rstrip(',') for line in output_lines]  # Remove trailing commas
        # print(processed_output)
        # print()


        ##checker
        select = 0
        output_node = output
        parsed_stuck_node = copy.deepcopy(parsed_circuit)
        parsed_stuck_node['outputs'] = [stuck_node]
        primary_input = parsed_stuck_node['inputs'][0]
        base_expressions = backtrack_for_stuck_node(parsed_circuit, primary_input,select,output_node)
        faulty_base_expression= backtrack_for_stuck_node(parsed_circuit,stuck_node,select,output_node)
        true_expression = base_expressions[0]
        print("output expressions:")
        print(true_expression)
        print("patterns that satisfy the given condition are:")
        print((variables),"rest all are x(don't cares)")
        if stuck_node in variables:
                # Replace occurrences of stuck_node with stuck_node_new
                stuck_node_regex = r'(?<!\w)' + re.escape(stuck_node) + r'(?!\w)'
                faulty_expression = re.sub(stuck_node_regex, f'{stuck_node}_new', faulty_base_expression[0])
                stuck_node = f"{stuck_node}_new"
        else:
            faulty_expression = faulty_base_expression[0]

        # print(faulty_expression)
        # print(len(processed_output))
        # Generate Verilog file
        if(len(processed_output)):
            generate_verilog(variables, processed_output, true_expression, faulty_expression, stuck_node, SA_0_1)
            compile_cmd = f"iverilog -o run generated_verilog.v"
            subprocess.run(compile_cmd, shell=True, check=True, text=True)

            # Run the compiled Verilog file
            run_cmd = "vvp run"
            result = subprocess.run(run_cmd, shell=True, check=True, text=True, stdout=subprocess.PIPE)
            raw_output = result.stdout.strip()  # Remove leading/trailing whitespace
            output_lines = raw_output.split("\n")  # Split into lines
            # print(output_lines)
            for i in range(0,len(output_lines)):
                print(output_lines[i])
        else:
            print("no pattern detected")

Boolean difference of sum = (A ^ B) & (((1 ^ Cin)) ^ ((0 ^ Cin)))
output expressions:
((A ^ B) ^ Cin)
patterns that satisfy the given condition are:
['Cin', 'A', 'B'] rest all are x(don't cares)
Test case = 001, True Output = 1, Faulty Output = 0
Test case = 010, True Output = 1, Faulty Output = 0
Test case = 101, True Output = 0, Faulty Output = 1
Test case = 110, True Output = 0, Faulty Output = 1
Boolean difference of carry = (A ^ B) & ((((A & B) | (1 & Cin))) ^ (((A & B) | (0 & Cin))))
output expressions:
((A & B) | ((A ^ B) & Cin))
patterns that satisfy the given condition are:
['Cin', 'A', 'B'] rest all are x(don't cares)
Test case = 101, True Output = 1, Faulty Output = 0
Test case = 110, True Output = 1, Faulty Output = 0
